# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/yMJC2/src/ase_conversions.jl:100
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/yMJC2/src/ase_conversions.jl:100


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/yMJC2/src/ase_conversions.jl:100
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298683019102                   -0.85    6.1    207ms
  2   -8.300225274409       -2.81       -1.25    2.1    102ms
  3   -8.300439428372       -3.67       -1.85    1.9    115ms
  4   -8.300459956298       -4.69       -2.58    1.0   90.1ms
  5   -8.300464270116       -5.37       -3.00    1.9    108ms
  6   -8.300464578961       -6.51       -3.19    1.6   99.4ms
  7   -8.300464620793       -7.38       -3.44    1.1   97.4ms
  8   -8.300464632481       -7.93       -3.64    1.5   93.4ms
  9   -8.300464637566       -8.29       -3.80    1.6   98.9ms
 10   -8.300464641635       -8.39       -4.05    1.0   88.1ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/yMJC2/src/ase_conversions.jl:100
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67027348342                   -0.70    5.9    493ms
  2   -16.67320286534       -2.53       -1.14    1.4    221ms
  3   -16.67923283400       -2.22       -1.83    3.4    297ms
  4   -16.67927425461       -4.38       -2.52    2.1    232ms
  5   -16.67928481810       -4.98       -3.07    2.2    269ms
  6   -16.67928611809       -5.89       -3.38    4.6    323ms
  7   -16.67928620376       -7.07       -3.96    1.1    196ms
  8   -16.67928622006       -7.79       -4.38    2.6    285ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/yMJC2/src/ase_conversions.jl:100
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32629305858                   -0.56    7.4    1.49s
  2   -33.33287848349       -2.18       -1.00    1.0    681ms
  3   -33.33412537030       -2.90       -1.73    4.2    963ms
  4   -33.33488681269       -3.12       -2.38    1.2    693ms
  5   -33.33692464072       -2.69       -2.59   11.5    1.36s
  6   -33.33693899045       -4.84       -2.93    3.4    812ms
  7   -33.33694334901       -5.36       -3.63    2.5    793ms
  8   -33.33694363672       -6.54       -3.75    6.4    1.18s
  9   -33.33694375983       -6.91       -4.18    1.0    637ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/yMJC2/src/ase_conversions.jl:100
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298716014950                   -0.85    5.2    224ms
  2   -8.300288315612       -2.80       -1.58    1.1   79.5ms
  3   -8.300382221975       -4.03       -2.31    3.0    104ms
  4   -8.300326010095   +   -4.25       -2.18    2.5    110ms
  5   -8.300463929138       -3.86       -3.46    1.0   77.1ms
  6   -8.300464517999       -6.23       -3.70   10.4    217ms
  7   -8.300464633295       -6.94       -4.13    1.1   91.2ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/yMJC2/src/ase_conversions.jl:100
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32801926525                   -0.56    7.4    1.83s
  2   -33.33322657271       -2.28       -1.28    1.5    621ms
  3   -31.58753950696   +    0.24       -0.94    6.5    1.23s
  4   -32.92480737076        0.13       -1.25    5.1    1.17s
  5   -33.26180000621       -0.47       -1.41    3.4    923ms
  6   -32.97248830377   +   -0.54       -1.28    3.5    929ms
  7   -33.32316648089       -0.46       -1.94    4.0    943ms
  8   -33.33508720867       -1.92       -2.37    1.6    647ms
  9   -33.33584442039       -3.12       -2.50    3.1    889ms
 10   -33.33668030341       -3.08       -2.66    2.1    742ms
 11   -33.33683105540       -3.82       -2.85    1.8    709ms
 12   -33.33692987588       -4.01 

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.